# Updated Model

Notebook by: Jieyao Wang


Coding up the model under the new setting; 
follow Andrew's previous setting

#### import packages

In [ ]:
# Required packages
import os
import sys
sys.path.append('./src')
from supportfunctions import *
sys.stdout.flush()

In [2]:
#### for now set the damage function to be the low damage case only

In [ ]:
# Damage function choices
damageSpec = 'Low'  # Choose among "High"(Weitzman), 'Low'(Nordhaus) and 'Weighted' (arithmeticAverage of the two)

if damageSpec == 'High':
    weight = 0.0
elif damageSpec == 'Low':
    weight = 1.0
else:
    weight = 0.5

xi_p =  1000  # Ambiguity Averse Paramter 

if xi_p < 1:
    aversespec = "Averse"
else:
    aversespec = 'Neutral'

smart_guess = False

In [ ]:
start_time = time.time()
    
McD = np.loadtxt('./data/TCRE_MacDougallEtAl2017_update.txt')
par_lambda_McD = McD / 1000

beta_f = np.mean(par_lambda_McD)  # Climate sensitivity parameter, MacDougall (2017)
sigma_f = np.var(par_lambda_McD, ddof = 1)  # varaiance of climate sensitivity parameters
lambda = 1.0 / sigma_f 

# Parameters as defined in the paper
delta = 0.01        
kappa = 0.032       
sigma_k = 0.0161
sigma_r = 0.0339 
alpha = 0.115000000000000
phi_0 = 0.0600
phi_1 = 16.666666666666668
mu_k = -0.034977443912449
psi_0 = 0.112733407891680
psi_1 = 0.142857142857143

# parameters for damage function settings
power = 2 
gamma_1 = 0.00017675
gamma_2 = 2. * 0.0022
gamma_2_plus = 2. * 0.0197
bar_gamma_2_plus = weight * 0 + (1 - weight) * gamma_2_plus

sigma_1 = 0
sigma_2 = 0
rho_12 = 0
F_bar = 2
crit = 2
F0 = 1

xi_d = -1 * (1 - kappa)
xi_k = -1 * (1 - kappa)

epsilon = 0.3
eta = 0.05
tol = 1e-8


R_min = 0
R_max = 9
F_min = 0
F_max = 4000
T_min = 0
T_max = 5
nR = 30
nF = 40
nT = 25
R = np.linspace(R_min, R_max, nR)
F = np.linspace(F_min, F_max, nF)
T = np.linspace(T_min, T_max, nT)

R = np.arange(R_min, R_max + hR, hR)
nR = len(R)
F = np.arange(F_min, F_max + hF, hF)
nF = len(F)
T = np.arange(T_min, T_max + hT, hT)
nT = len(T)

(R_mat, F_mat, T_mat) = np.meshgrid(R,F,T, indexing = 'ij')
stateSpace = np.hstack([R_mat.reshape(-1,1,order = 'F'),F_mat.reshape(-1,1,order = 'F'),T_mat.reshape(-1,1,order = 'F')])

quadrature = 'legendre'
n = 30
a = beta_f - 5 * np.sqrt(sigma_f)
b = beta_f + 5 * np.sqrt(sigma_f)

v0 = kappa * R_mat + (1-kappa) * K_mat # - sigma_f * F_mat

FC_Err = 1
episode = 0

    
while episode == 0 or FC_Err > tol:
    vold = v0.copy()
    v0_dr = finiteDiff(v0,0,1,hR) 
    v0_df = finiteDiff(v0,1,1,hF)
    v0_dk = finiteDiff(v0,2,1,hK)

    v0_drr = finiteDiff(v0,0,2,hR)
    v0_drr[v0_dr < 1e-16] = 0
    v0_dr[v0_dr < 1e-16] = 1e-16
    v0_dff = finiteDiff(v0,1,2,hF)
    v0_dkk = finiteDiff(v0,2,2,hK)
    if episode > 2000:
        epsilon = 0.1
    elif episode > 1000:
        epsilon = 0.2
    else:
        pass

    if episode == 0:
        # First time into the loop
        B1 = v0_dr - xi_d * (gamma_1 + gamma_2 * F_mat * beta_f + gamma_2_plus * (F_mat * beta_f - F_bar) ** (power - 1) * \
                             (F_mat >= (crit / beta_f))) * beta_f * np.exp(R_mat) - v0_df * np.exp(R_mat)
        C1 = - delta * kappa
        e = -C1 / B1
        e_hat = e
        Acoeff = np.ones(R_mat.shape)
        Bcoeff = ((delta * (1 - kappa) * phi_1 + phi_0 * phi_1 * v0_dk) * δ * (1 - κ) / (v0_dr * ψ0 * 0.5) * np.exp(0.5 * (R_mat - K_mat))) / (δ * (1 - κ) * ϕ1)
        Ccoeff = -α  - 1 / ϕ1
        j = ((-Bcoeff + np.sqrt(Bcoeff ** 2 - 4 * Acoeff * Ccoeff)) / (2 * Acoeff)) ** 2
        i = α - j - (δ * (1 - κ)) / (v0_dr * ψ0 * 0.5) * j ** 0.5 * np.exp(0.5 * (R_mat - K_mat))
        q = δ * (1 - κ) / (α - i - j)
    else:
        e_hat = e_star
        
### no cobweb or i or j at all


    a1 = np.zeros(R_mat.shape)
    b1 = xi_d * e_hat * np.exp(R_mat) * γ1
    c1 = 2 * xi_d * e_hat * np.exp(R_mat) * F_mat * γ2 
    λ̃1 = λ + c1 / ξp
    β̃1 = β𝘧 - c1 * β𝘧 / (ξp * λ̃1) -  b1 /  (ξp * λ̃1)
    I1 = a1 - 0.5 * np.log(λ) * ξp + 0.5 * np.log(λ̃1) * ξp + 0.5 * λ * β𝘧 ** 2 * ξp - 0.5 * λ̃1 * (β̃1) ** 2 * ξp
    R1 = 1 / ξp * (I1 - (a1 + b1 * β̃1 + c1 / 2 * β̃1 ** 2 + c1 / 2 / λ̃1))
    J1_without_e = xi_d * (γ1 * β̃1 + γ2 * F_mat * (β̃1 ** 2 + 1 / λ̃1)) * np.exp(R_mat)

    π̃1 = weight * np.exp(-1 / ξp * I1)

    # Step (2), solve minimization problem in HJB and calculate drift distortion
    # See remark 2.1.3 for more details
    def scale_2_fnc(x):
        return np.exp(-1 / ξp * xi_d * (γ1 * x + γ2 * x ** 2 * F_mat + γ2_plus * x * (x * F_mat - F̄) ** (power - 1) * ((x * F_mat - F̄) >= 0)) * np.exp(R_mat) * e_hat)  * norm.pdf(x,β𝘧,np.sqrt(σᵦ))

    scale_2 = quad_int(scale_2_fnc, a, b, n, 'legendre')

    def q2_tilde_fnc(x):
        return np.exp(-1 / ξp * xi_d * (γ1 * x + γ2 * x ** 2 * F_mat + γ2_plus * x * (x * F_mat - F̄) ** (power - 1) * ((x * F_mat - F̄) >= 0)) * np.exp(R_mat) * e_hat) / scale_2

    I2 = -1 * ξp * np.log(scale_2)

    def J2_without_e_fnc(x):
        return xi_d * np.exp(R_mat) * q2_tilde_fnc(x) * (γ1 * x + γ2 * F_mat * x ** 2 + γ2_plus * x * (x * F_mat - F̄) ** (power - 1) * ((x * F_mat - F̄) >= 0)) * norm.pdf(x,β𝘧,np.sqrt(σᵦ))

    J2_without_e = quad_int(J2_without_e_fnc, a, b, n, 'legendre')
    J2_with_e = J2_without_e * e_hat

    R2 = (I2 - J2_with_e) / ξp
    π̃2 = (1 - weight) * np.exp(-1 / ξp * I2)
    π̃1_norm = π̃1 / (π̃1 + π̃2)
    π̃2_norm = 1 - π̃1_norm

    # step 4 (b) updating e based on first order conditions
    expec_e_sum = (π̃1_norm * J1_without_e + π̃2_norm * J2_without_e)

    B1 = v0_dr - v0_df * np.exp(R_mat) - expec_e_sum
    C1 = -δ * κ
    e = -C1 / B1
    e_star = e

    J1 = J1_without_e * e_star
    J2 = J2_without_e * e_star

    # Step (3) calculating implied entropies
    I_term = -1 * ξp * np.log(π̃1 + π̃2)

    R1 = (I1 - J1) / ξp
    R2 = (I2 - J2) / ξp
    
    # Step (5) solving for adjusted drift
    drift_distort = (π̃1_norm * J1 + π̃2_norm * J2)

    if weight == 0 or weight == 1:
        RE = π̃1_norm * R1 + π̃2_norm * R2
    else:
        RE = π̃1_norm * R1 + π̃2_norm * R2 + π̃1_norm * np.log(
            π̃1_norm / weight) + π̃2_norm * np.log(π̃2_norm / (1 - weight))

    RE_total = ξp * RE

    # Step (6) and (7) Formulating HJB False Transient parameters
    # See remark 2.1.4 for more details
    A = -δ * np.ones(R_mat.shape)
    B_r = -e_star + ψ0 * (j ** ψ1) * np.exp(ψ1 * (K_mat - R_mat)) - 0.5 * (σ𝘳 ** 2)
    B_f = e_star * np.exp(R_mat)
    B_k = μk + ϕ0 * np.log(1 + i * ϕ1) - 0.5 * (σ𝘬 ** 2)
    C_rr = 0.5 * σ𝘳 ** 2 * np.ones(R_mat.shape)
    C_ff = np.zeros(R_mat.shape)
    C_kk = 0.5 * σ𝘬 ** 2 * np.ones(R_mat.shape)
    D = δ * κ * np.log(e_star) + δ * κ * R_mat + δ * (1 - κ) * (np.log(α - i - j) + K_mat) + drift_distort + RE_total # + I_term 

    # Step (8) solving linear system using a conjugate-gradient method in C++
    # See remark 2.1.5, 2.1.6 for more details
    out = PDESolver(stateSpace, A, B_r, B_f, B_k, C_rr, C_ff, C_kk, D, v0, ε, solverType = 'False Transient')

    out_comp = out[2].reshape(v0.shape,order = "F")
    
    # Calculating PDE error and False Transient error
    PDE_rhs = A * v0 + B_r * v0_dr + B_f * v0_df + B_k * v0_dk + C_rr * v0_drr + C_kk * v0_dkk + C_ff * v0_dff + D
    PDE_Err = np.max(abs(PDE_rhs))
    FC_Err = np.max(abs((out_comp - v0)))
    if episode % 100 == 0:
        print("Episode {:d}: PDE Error: {:.10f}; False Transient Error: {:.10f}; Iterations: {:d}; CG Error: {:.10f}" .format(episode, PDE_Err, FC_Err, out[0], out[1]))
    episode += 1
    
    # step 9: keep iterating until convergence
    v0 = out_comp

print("Episode {:d}: PDE Error: {:.10f}; False Transient Error: {:.10f}; Iterations: {:d}; CG Error: {:.10f}" .format(episode, PDE_Err, FC_Err, out[0], out[1]))
print("--- %s seconds ---" % (time.time() - start_time))

